In [40]:
from dimod import ConstrainedQuadraticModel, Real, Integer, Binary, quicksum

def mps2cmq(filename):
    ### read the mps file ###
    mpsFile = open(filename, "r")
    mpsLine = mpsFile.readlines()
    mpsLine = [s.replace('\n', '') for s in mpsLine]

    readRow, readCol, readInt, readRHS, readBnd, readObj = False, False, False, False, False, False
    equations, bi_variables, int_variables, real_variables, objEq = {}, set(), set(), set(), None
    LHSeq, RHSeq, upBound, lowBound = {}, {}, {}, {}

    for i in range(len(mpsLine)):
        string = mpsLine[i]  
        if  string == 'ROWS':
            readRow = True
        elif string == 'COLUMNS':
            readRow = False
            readCol = True
            LHSeq = {k: [] for k in equations.keys()}
            RHSeq = {k: [] for k in equations.keys()}
        elif string == 'RHS':
            readCol = False
            readRHS = True
        elif string == 'BOUNDS':
            readRHS = False
            readBnd = True

        if readRow and string != 'ROWS':
            relation, eq = string.split()
            equations[eq] = relation
            if not readObj and relation == 'N': ObjEq = eq
        
        if readCol and string != 'COLUMNS':
            strList = string.split()
            if strList[1] == "\'MARKER\'" and strList[2] == "\'INTORG\'":
                readInt = True
                continue
            elif strList[1] == "\'MARKER\'" and strList[2] == "\'INTEND\'":
                readInt = False
                continue

            var = strList[0]
            if readInt:
                int_variables.add(var)
            else:
                real_variables.add(var)
            for i in range(0, len(strList)//2):
                LHSeq[strList[i*2+1]].append((var, strList[i*2+2]))
        
        if readRHS and string != 'RHS':
            strList = string.split()
            for i in range(0, len(strList)//2):
                RHSeq[strList[i*2+1]] = strList[i*2+2]
        
        if readBnd and string != 'BOUNDS':
            strList = string.split()
            if strList[0] == 'UP':
                upBound[strList[2]] = strList[3]
            elif strList[0] == 'LO':
                lowBound[strList[2]] = strList[3]

    mpsFile.close()

    ### convert into cqm ###
    cqm = ConstrainedQuadraticModel()

    # variables
    for s in upBound:
        if upBound[s] == '1' and s in int_variables:
            bi_variables.add(s)
            int_variables.remove(s)

    x = {v: Real(v) for v in real_variables}
    x_int = {v: Integer(v) for v in int_variables}
    x_bi = {v: Binary(v) for v in bi_variables}
    x.update(x_int)
    x.update(x_bi)

    # the objective function and constraint equations
    for eq in equations:
        currLHS = quicksum(x[v]*float(coef) for v, coef in LHSeq[eq])

        if eq == ObjEq:
            cqm.set_objective(currLHS)
            continue

        RHSvalue = float(RHSeq[eq]) if RHSeq[eq] else 0.0
        if equations[eq] == 'E':
            cqm.add_constraint(currLHS == RHSvalue, label=eq)
        elif equations[eq] == 'G':
            cqm.add_constraint(currLHS >= RHSvalue, label=eq)
        elif equations[eq] == 'L':
            cqm.add_constraint(currLHS <= RHSvalue, label=eq)

    # constraints on the variables
    for k in upBound.keys():
         if k not in bi_variables: cqm.set_upper_bound(k, float(upBound[k]))
    for k in lowBound.keys():
        cqm.set_lower_bound(k, float(lowBound[k]))
    
    return cqm

In [41]:
from dwave.system import LeapHybridCQMSampler

# read mps file
# cqm1 = mps2cmq('testprob.mps')
# cqm1 = mps2cmq('markshare_4_0.mps')
cqm1 = mps2cmq('50v-10.mps')

In [52]:
# run the solver
sampler = LeapHybridCQMSampler()
sampleset = sampler.sample_cqm(cqm1, label='50v-10')

In [55]:
# analyze the results
feasible_sampleset = sampleset.filter(lambda d: d.is_feasible)
feasible_sampleset.first.energy

6600.5000042021275

In [22]:
for v in cqm1.variables:
    print(v + ": " + str(cqm1.vartype(v)))

ARY
x1343: Vartype.BINARY
x1344: Vartype.BINARY
x1345: Vartype.BINARY
x1346: Vartype.BINARY
x1347: Vartype.INTEGER
x1348: Vartype.BINARY
x1349: Vartype.BINARY
x1350: Vartype.BINARY
x1351: Vartype.BINARY
x1352: Vartype.BINARY
x1353: Vartype.BINARY
x1354: Vartype.BINARY
x1355: Vartype.BINARY
x1356: Vartype.INTEGER
x1357: Vartype.BINARY
x1358: Vartype.BINARY
x1359: Vartype.BINARY
x1360: Vartype.BINARY
x1361: Vartype.BINARY
x1362: Vartype.BINARY
x1363: Vartype.BINARY
x1364: Vartype.BINARY
x1365: Vartype.INTEGER
x1366: Vartype.BINARY
x1367: Vartype.BINARY
x1368: Vartype.BINARY
x1369: Vartype.BINARY
x1370: Vartype.BINARY
x1371: Vartype.BINARY
x1372: Vartype.BINARY
x1373: Vartype.BINARY
x1374: Vartype.INTEGER
x1375: Vartype.BINARY
x1376: Vartype.BINARY
x1377: Vartype.BINARY
x1378: Vartype.BINARY
x1379: Vartype.BINARY
x1380: Vartype.BINARY
x1381: Vartype.BINARY
x1382: Vartype.BINARY
x1383: Vartype.INTEGER
x1384: Vartype.BINARY
x1385: Vartype.BINARY
x1386: Vartype.BINARY
x1387: Vartype.BINARY
x

In [42]:
cqm1.constraint_labels

Variables(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c69', 'c70', 'c71', 'c72', 'c73', 'c74', 'c75', 'c76', 'c77', 'c78', 'c79', 'c80', 'c81', 'c82', 'c83', 'c84', 'c85', 'c86', 'c87', 'c88', 'c89', 'c90', 'c91', 'c92', 'c93', 'c94', 'c95', 'c96', 'c97', 'c98', 'c99', 'c100', 'c101', 'c102', 'c103', 'c104', 'c105', 'c106', 'c107', 'c108', 'c109', 'c110', 'c111', 'c112', 'c113', 'c114', 'c115', 'c116', 'c117', 'c118', 'c119', 'c120', 'c121', 'c122', 'c123', 'c124', 'c125', 'c126', 'c127', 'c128', 'c129', 'c130', 'c131', 'c132', 'c133', 'c134', 'c135', 'c136', 'c137', '

In [51]:
cqm1.constraints['c35']

Eq(ConstraintView({'x67': -1.0, 'x250': 1.0, 'x151': -1.0, 'x334': 1.0, 'x103': -1.0, 'x286': 1.0, 'x159': -1.0, 'x342': 1.0, 'x68': 1.0, 'x134': 1.0, 'x251': -1.0, 'x317': -1.0}, {}, 0.0, {'x67': 'REAL', 'x250': 'REAL', 'x151': 'REAL', 'x334': 'REAL', 'x103': 'REAL', 'x286': 'REAL', 'x159': 'REAL', 'x342': 'REAL', 'x68': 'REAL', 'x134': 'REAL', 'x251': 'REAL', 'x317': 'REAL'}), -91.0)